# Module 1.1: 2D Convolution - 二維卷積

卷積是影像處理和 CNN 的核心操作。這個 notebook 會讓你從零開始理解並實作卷積。

## 學習目標

1. 理解卷積的直覺（滑動視窗）
2. 實作 naive 版本（雙重迴圈）
3. 理解 padding 和 stride
4. 實作向量化版本

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
sys.path.append('../..')  # 添加專案根目錄

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

---
## Part 1: 卷積的直覺

### 什麼是卷積？

卷積就是**滑動視窗 + 加權和**：

1. 把一個小的「kernel（核）」放在影像上
2. 計算 kernel 覆蓋區域的加權和
3. 滑動 kernel，對每個位置重複計算

### 數學定義

$$\text{output}[i, j] = \sum_m \sum_n \text{input}[i+m, j+n] \times \text{kernel}[m, n]$$

### Kernel 的作用

不同的 kernel 可以做不同的事：
- **平均 kernel**：模糊
- **Gaussian kernel**：高斯模糊
- **Sobel kernel**：邊緣偵測
- **Sharpen kernel**：銳化

In [ ]:
# 視覺化卷積過程
def visualize_convolution_step(image, kernel, position):
    """
    視覺化卷積的一個步驟
    """
    i, j = position
    kh, kw = kernel.shape
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    # 1. 原始影像
    ax = axes[0]
    ax.imshow(image, cmap='gray')
    # 畫出 kernel 位置
    rect = plt.Rectangle((j-0.5, i-0.5), kw, kh, 
                          fill=False, edgecolor='red', linewidth=2)
    ax.add_patch(rect)
    ax.set_title('Input Image\n(紅框 = kernel 位置)')
    
    # 2. 被 kernel 覆蓋的區域
    ax = axes[1]
    region = image[i:i+kh, j:j+kw]
    ax.imshow(region, cmap='gray')
    for ii in range(kh):
        for jj in range(kw):
            ax.text(jj, ii, f'{region[ii,jj]:.0f}', ha='center', va='center', fontsize=12)
    ax.set_title('覆蓋的區域')
    
    # 3. Kernel
    ax = axes[2]
    ax.imshow(kernel, cmap='RdBu_r')
    for ii in range(kh):
        for jj in range(kw):
            ax.text(jj, ii, f'{kernel[ii,jj]:.2f}', ha='center', va='center', fontsize=12)
    ax.set_title('Kernel')
    
    # 4. 計算結果
    ax = axes[3]
    result = np.sum(region * kernel)
    ax.text(0.5, 0.5, f'Σ(region × kernel)\n= {result:.2f}', 
            ha='center', va='center', fontsize=16, transform=ax.transAxes)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title(f'Output[{i}, {j}]')
    
    plt.tight_layout()
    plt.show()

# 範例
image = np.array([
    [10, 20, 30, 40, 50],
    [60, 70, 80, 90, 100],
    [110, 120, 130, 140, 150],
    [160, 170, 180, 190, 200],
    [210, 220, 230, 240, 250]
], dtype=float)

kernel = np.array([
    [1, 0, -1],
    [2, 0, -2],
    [1, 0, -1]
], dtype=float) / 4  # Sobel X (normalized)

visualize_convolution_step(image, kernel, (1, 1))

---
## Part 2: 實作 Naive Convolution

### 練習 1: 實作基本卷積（無 padding）

**提示**：
1. 輸出尺寸 = 輸入尺寸 - kernel 尺寸 + 1
2. 用雙重迴圈遍歷每個輸出位置
3. 對每個位置，取出對應的 patch，和 kernel 做元素相乘再加總

In [ ]:
def conv2d_naive(image, kernel):
    """
    最簡單的 2D 卷積實作（無 padding）
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
        輸入灰階影像
    kernel : np.ndarray, shape (kH, kW)
        卷積核
    
    Returns
    -------
    output : np.ndarray, shape (H - kH + 1, W - kW + 1)
        卷積結果
    
    Example
    -------
    >>> img = np.ones((5, 5))
    >>> kernel = np.ones((3, 3)) / 9
    >>> out = conv2d_naive(img, kernel)
    >>> out.shape
    (3, 3)
    """
    # 解答：
    H, W = image.shape
    kH, kW = kernel.shape
    
    # 輸出尺寸
    out_H = H - kH + 1
    out_W = W - kW + 1
    
    output = np.zeros((out_H, out_W))
    
    for i in range(out_H):
        for j in range(out_W):
            # 取出被 kernel 覆蓋的區域
            patch = image[i:i+kH, j:j+kW]
            # 元素相乘再加總
            output[i, j] = np.sum(patch * kernel)
    
    return output

In [ ]:
# 測試
test_image = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
], dtype=float)

# 平均 kernel
avg_kernel = np.ones((3, 3)) / 9

result = conv2d_naive(test_image, avg_kernel)
print("Input shape:", test_image.shape)
print("Kernel shape:", avg_kernel.shape)
print("Output shape:", result.shape)
print("\nOutput:")
print(result)

# 驗證：中心點應該是 (6+7+8+10+11+12+14+15+16)/9 = 11
print(f"\n中心點 = {result[0,0]:.4f}（應該 ≈ 平均值）")

---
## Part 3: Padding

### 為什麼需要 Padding？

1. **保持輸出尺寸**：沒有 padding，輸出會縮小
2. **保留邊緣資訊**：邊緣像素被用到的次數較少

### Padding 類型

- **valid**：不 padding，輸出縮小
- **same**：padding 使輸出和輸入一樣大
- **full**：padding 使每個輸入像素都被完全覆蓋

### 練習 2: 實作帶 padding 的卷積

In [ ]:
def pad_image(image, pad_h, pad_w, mode='constant', constant_value=0):
    """
    對影像做 padding
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
    pad_h : int
        上下各 pad 多少
    pad_w : int
        左右各 pad 多少
    mode : str
        'constant' (填 0) 或 'reflect' (鏡像)
    """
    # 解答：
    if mode == 'constant':
        return np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), 
                      mode='constant', constant_values=constant_value)
    elif mode == 'reflect':
        return np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')
    else:
        raise ValueError(f"Unknown mode: {mode}")


def conv2d(image, kernel, padding='valid'):
    """
    2D 卷積，支援 padding
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
    kernel : np.ndarray, shape (kH, kW)
    padding : str or int
        'valid': 不 padding
        'same': 輸出和輸入一樣大
        int: 指定 padding 大小
    """
    # 解答：
    kH, kW = kernel.shape
    
    if padding == 'valid':
        pad_h, pad_w = 0, 0
    elif padding == 'same':
        # 計算需要多少 padding 使輸出尺寸 = 輸入尺寸
        pad_h = (kH - 1) // 2
        pad_w = (kW - 1) // 2
    elif isinstance(padding, int):
        pad_h = pad_w = padding
    else:
        raise ValueError(f"Unknown padding: {padding}")
    
    # Pad the image
    if pad_h > 0 or pad_w > 0:
        image = pad_image(image, pad_h, pad_w)
    
    # 用 naive 方法做卷積
    return conv2d_naive(image, kernel)

In [ ]:
# 測試 padding
test_image = np.arange(1, 26).reshape(5, 5).astype(float)
kernel = np.ones((3, 3)) / 9

print("Input shape:", test_image.shape)
print("\nvalid padding:")
out_valid = conv2d(test_image, kernel, padding='valid')
print(f"  Output shape: {out_valid.shape}")

print("\nsame padding:")
out_same = conv2d(test_image, kernel, padding='same')
print(f"  Output shape: {out_same.shape}")

---
## Part 4: 用真實影像測試

讓我們在真實影像上測試我們的卷積實作。

In [ ]:
# 創建一個簡單的測試影像（如果沒有真實影像）
def create_test_image(size=128):
    """創建一個有邊緣的測試影像"""
    img = np.zeros((size, size))
    
    # 添加一個白色方塊
    s = size // 4
    img[s:3*s, s:3*s] = 255
    
    # 添加一個漸層
    gradient = np.linspace(0, 255, size)
    img[:, :size//8] = gradient[:, np.newaxis][:size, :]
    
    # 添加一些圓形
    y, x = np.ogrid[:size, :size]
    center = (size * 3 // 4, size * 3 // 4)
    radius = size // 8
    mask = (x - center[0])**2 + (y - center[1])**2 <= radius**2
    img[mask] = 200
    
    return img

test_img = create_test_image(128)

plt.figure(figsize=(6, 6))
plt.imshow(test_img, cmap='gray')
plt.title('Test Image')
plt.colorbar()
plt.show()

In [ ]:
# 定義各種 kernel
kernels = {
    'Identity': np.array([[0, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]], dtype=float),
    
    'Box Blur 3x3': np.ones((3, 3)) / 9,
    
    'Box Blur 5x5': np.ones((5, 5)) / 25,
    
    'Sobel X': np.array([[-1, 0, 1],
                         [-2, 0, 2],
                         [-1, 0, 1]], dtype=float),
    
    'Sobel Y': np.array([[-1, -2, -1],
                         [0, 0, 0],
                         [1, 2, 1]], dtype=float),
    
    'Sharpen': np.array([[0, -1, 0],
                         [-1, 5, -1],
                         [0, -1, 0]], dtype=float),
}

# 應用各種 kernel
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for ax, (name, kernel) in zip(axes, kernels.items()):
    result = conv2d(test_img, kernel, padding='same')
    
    # 對於 Sobel，取絕對值以便顯示
    if 'Sobel' in name:
        result = np.abs(result)
    
    ax.imshow(result, cmap='gray')
    ax.set_title(name)
    ax.axis('off')

plt.tight_layout()
plt.show()

---
## Part 5: Stride

Stride 控制 kernel 每次滑動多少：
- stride=1：每次滑動 1 格（預設）
- stride=2：每次滑動 2 格，輸出縮小一半

### 練習 3: 實作帶 stride 的卷積

In [ ]:
def conv2d_full(image, kernel, padding='valid', stride=1):
    """
    完整的 2D 卷積，支援 padding 和 stride
    
    Parameters
    ----------
    image : np.ndarray, shape (H, W)
    kernel : np.ndarray, shape (kH, kW)
    padding : str or int
    stride : int
    
    Returns
    -------
    output : np.ndarray
    """
    # 解答：
    kH, kW = kernel.shape
    
    # 處理 padding
    if padding == 'valid':
        pad_h, pad_w = 0, 0
    elif padding == 'same':
        pad_h = (kH - 1) // 2
        pad_w = (kW - 1) // 2
    elif isinstance(padding, int):
        pad_h = pad_w = padding
    else:
        raise ValueError(f"Unknown padding: {padding}")
    
    # Pad
    if pad_h > 0 or pad_w > 0:
        image = pad_image(image, pad_h, pad_w)
    
    H, W = image.shape
    
    # 計算輸出尺寸
    out_H = (H - kH) // stride + 1
    out_W = (W - kW) // stride + 1
    
    output = np.zeros((out_H, out_W))
    
    for i in range(out_H):
        for j in range(out_W):
            # 計算在 padded image 上的位置
            ii = i * stride
            jj = j * stride
            patch = image[ii:ii+kH, jj:jj+kW]
            output[i, j] = np.sum(patch * kernel)
    
    return output

In [ ]:
# 測試不同的 stride
kernel = np.ones((3, 3)) / 9

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

strides = [1, 2, 3, 4]
for ax, s in zip(axes, strides):
    result = conv2d_full(test_img, kernel, padding='same', stride=s)
    ax.imshow(result, cmap='gray')
    ax.set_title(f'Stride = {s}\nOutput shape: {result.shape}')

plt.tight_layout()
plt.show()

---
## 練習題：整合練習

### 練習 4: 計算邊緣強度

使用 Sobel X 和 Sobel Y 計算邊緣強度：

$$\text{magnitude} = \sqrt{G_x^2 + G_y^2}$$

In [ ]:
def compute_edge_magnitude(image):
    """
    計算邊緣強度
    
    1. 用 Sobel X 計算 x 方向梯度
    2. 用 Sobel Y 計算 y 方向梯度
    3. 計算強度 = sqrt(Gx² + Gy²)
    """
    # 解答：
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]], dtype=float)
    
    sobel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]], dtype=float)
    
    Gx = conv2d_full(image, sobel_x, padding='same')
    Gy = conv2d_full(image, sobel_y, padding='same')
    
    magnitude = np.sqrt(Gx**2 + Gy**2)
    
    return magnitude, Gx, Gy

In [ ]:
# 測試邊緣偵測
magnitude, Gx, Gy = compute_edge_magnitude(test_img)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_img, cmap='gray')
axes[0].set_title('Original')

axes[1].imshow(np.abs(Gx), cmap='gray')
axes[1].set_title('|Gx| (Horizontal edges)')

axes[2].imshow(np.abs(Gy), cmap='gray')
axes[2].set_title('|Gy| (Vertical edges)')

axes[3].imshow(magnitude, cmap='gray')
axes[3].set_title('Edge Magnitude')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

---
## Summary

這個 notebook 你學到了：

1. **卷積的直覺**：滑動視窗 + 加權和
2. **基本實作**：雙重迴圈遍歷每個輸出位置
3. **Padding**：valid（不 pad）vs same（保持尺寸）
4. **Stride**：控制滑動步長，可以縮小輸出
5. **應用**：不同 kernel 做模糊、邊緣偵測、銳化

### 卷積在 ML 中的應用：

| 應用 | 說明 |
|------|------|
| CNN | 學習 kernel 來擷取特徵 |
| 影像濾波 | 去噪、銳化 |
| 邊緣偵測 | Sobel, Canny |
| 特徵擷取 | HOG, SIFT |

---

**下一個 notebook**: `02_filters.ipynb` - 常見濾波器